In [1]:
# Test code
depth = 510
target_x, target_y = 10, 10

In [2]:
import re

input = open("22.txt").read()
groups = re.match("depth: (\d+)\ntarget: (\d+),(\d+)", input).groups()
depth, target_x, target_y = (int(x.replace(",", "")) for x in groups)
depth, target_x, target_y

(6084, 14, 709)

In [3]:
from functools import lru_cache


@lru_cache(None)
def geo_ix(x, y):
    if x == y == 0:
        return 0
    if x == target_x and y == target_y:
        return 0
    if y == 0:
        return x * 16807
    if x == 0:
        return y * 48271
    return erosion_level(x - 1, y) * erosion_level(x, y - 1)

In [4]:
def erosion_level(x, y):
    return (geo_ix(x, y) + depth) % 20183

In [5]:
@lru_cache(None)
def regtype(x, y):
    return erosion_level(x, y) % 3

# Part I

In [6]:
sum(regtype(x, y) for x in range(target_x + 1) for y in range(target_y + 1))

10603

# Part II

region type ```Rock, wet, narrow``` is numbered ```0,1,2```.  
Now we also number the tools ```neither, torch, climbing gear``` ```0,1,2```.  
That means a tool is not allowed if it's equal to the region type

In [7]:
import heapq

heapq always returns the element with the lowest value. since we keep on adding al neighbours, there is no way we can get to the first element in a faster way (as it has to take another route, but there is no shorter route left)

In [8]:
%%time
heap = [(0, (0, 0, 1))]  # mins, (x, y, tool)
best = dict()
while True:
    mins, key = heapq.heappop(heap)
    if key in best:
        assert best[key] <= mins
        continue
    if key == (target_x, target_y, 1):
        print(f"answer={mins}")
        break
    best[key] = mins

    (x, y, tool) = key
    for newtool in range(3):
        if newtool != regtype(x, y) and tool != newtool:
            heapq.heappush(heap, (mins + 7, (x, y, newtool)))

    for dx, dy in ((0, 1), (1, 0), (-1, 0), (0, -1)):
        (newx, newy) = (x + dx, y + dy)
        if newx >= 0 and newy >= 0 and regtype(newx, newy) != tool:
            heapq.heappush(heap, (mins + 1, (newx, newy, tool)))

answer=952
Wall time: 25.4 s
